In [ ]:
import os
import cv2
import math
import imutils
import numpy as np
import pandas as pd
from copy import deepcopy
from random import randint
from PIL.ImageShow import show
from PIL import Image


In [ ]:
def LoadDataSet(dataSetFolderPath: str):
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    imageSize = []
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        imageSize.append([image.shape[0], image.shape[1]])

    imageSizeDF = pd.DataFrame(imageSize, columns=["width", "height"])
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique()
    uniquewidth = imageSizeDF["width"].unique().tolist()
    uniqueheight = imageSizeDF["height"].unique().tolist()
    print(len(uniquewidth), uniquewidth)
    print(len(uniqueheight), uniqueheight)


In [ ]:

def SignDetection(image):
    try:
        grayScale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except:
        grayScale = image
    bilateralFilter = cv2.bilateralFilter(grayScale, 11, 17, 17)

    imageMedian = np.median(image)
    lowerThreshold = max(0, (0.7 * imageMedian))
    upperThreshold = min(255, (0.7 * imageMedian))

    cannyEdged = cv2.Canny(bilateralFilter, lowerThreshold, upperThreshold)
    (cnts, _) = cv2.findContours(cannyEdged,
                                 cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:]
    maxArea = 0
    SignsDetected = []
    for c in cnts:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * perimeter, True)
        if (len(approx) == 4):
            signArea = approx
            x, y, w, h = cv2.boundingRect(signArea)
            area = w*h
            if(area >= maxArea):
                maxArea = area
                if([x, y, w, h] not in SignsDetected):
                    SignsDetected.append([x, y, w, h])
    return SignsDetected


In [ ]:
def SaveImage(name: str, image: np, displayImage: bool = False) -> None:
    im = Image.fromarray(image.astype(np.uint8))
    im.save(name+".png")
    if(displayImage):
        show(im, name)


In [ ]:
def GrayScaleConverstion(image: np, greyScalefilter: np = np.array([0.2989, 0.5870, 0.1140])) -> np:
    imagecopy = deepcopy(image)
    greyScaleImage = np.dot(imagecopy[:, :, :3], greyScalefilter)
    return greyScaleImage


In [ ]:
def AddGaussianNoise(image: np, mean: float, std: float):
    noise = np.random.normal(mean, std, size=image.shape)
    noisyImage = image + noise
    return noisyImage


def AddSaltPepperNoise(image: np, percent: int) -> np:
    saltPepperImage = np.copy(image)
    saltPepperCount = (((image.shape[0]*image.shape[1])//100)*percent)//2
    for i in range(saltPepperCount):
        saltPepperPixel = [
            randint(0, image.shape[0])-1, randint(0, image.shape[1])-1]
        saltPepperImage[saltPepperPixel[0], saltPepperPixel[1]] = 0
        saltPepperPixel = [
            randint(0, image.shape[0])-1, randint(0, image.shape[1])-1]
        saltPepperImage[saltPepperPixel[0], saltPepperPixel[1]] = 255
    return saltPepperImage


In [ ]:
def MedianFilter(image: np, k: int) -> np:
    outputImage = np.zeros(image.shape)
    padding = (k//2)*2
    paddedImage = np.zeros((image.shape[0]+padding, image.shape[1]+padding))
    # print(image.shape, paddedImage.shape)
    paddedImage[(padding//2):paddedImage.shape[0] - (padding//2),
                (padding//2):paddedImage.shape[1] - (padding//2)] = image
    noRows, noCols = image.shape
    for i in range(noRows):
        for j in range(noCols):
            try:
                outputImage[i, j] = np.median(paddedImage[i:i+k, j:j+k])
            except:
                print("error")
                # outputImage[i,j] = image[i,j]
    return outputImage


In [ ]:
def FindNorm(k: float, std: int, mean: int = 0):
    denominator = (math.sqrt(2 * math.pi)*std)
    expo = -((k**2) / 2*(std**2))
    return (math.e**expo)/denominator


def GenerateGaussianKernel(size: int, std: int = 1) -> np:
    OneDKernel = np.linspace(-(size // 2), size // 2, size)
    for i in range(size):
        OneDKernel[i] = FindNorm(OneDKernel[i], std)
    twoDKernel = np.outer(OneDKernel, OneDKernel)
    twoDKernel *= 1.0 / twoDKernel.max()
    return twoDKernel


def GenerateBoxKernel(size: int) -> np:
    boxFilter = np.zeros((size, size))
    boxFilter += 1
    boxFilter = boxFilter / np.sum(boxFilter)
    return boxFilter


In [ ]:
def Convolve(image: np, kernel: np) -> np:
    outputImage = np.zeros(image.shape)
    noRows, noCols = image.shape
    noKRows, noKCols = kernel.shape
    paddingHeight, paddingWidth = (noKRows//2)*2, (noKCols//2)*2
    paddedImage = np.zeros((noRows + paddingHeight, noCols + paddingWidth))
    paddedImage[(paddingHeight//2):paddedImage.shape[0] - (paddingHeight//2),
                (paddingWidth//2):paddedImage.shape[1] - (paddingWidth//2)] = image
    for i in range(noRows):
        for j in range(noCols):
            outputImage[i, j] = np.sum(
                kernel*paddedImage[i:i+noKRows, j:j+noKCols])
    return outputImage


In [ ]:
allImageNames = os.listdir("./Traffic Signs")
mean = 0
greyScaleFilter = np.array([0.2989, 0.5870, 0.1140])
boxKernel3x3 = GenerateBoxKernel(3)
boxKernel5x5 = GenerateBoxKernel(5)
gaussianKernel5x5 = GenerateGaussianKernel(5, 3)
boxKernel9x9 = GenerateBoxKernel(9)
gaussianKernel9x9 = GenerateGaussianKernel(9, 5)
gaussianKernel3x3 = GenerateGaussianKernel(3, 5)

for imageName in allImageNames:
    image = cv2.imread("./Traffic Signs/"+imageName)
    imageName_ = imageName.split('.')[0]
    imageName_ = ''.join(c for c in imageName_ if c.isalnum())
    print("Working on", imageName_)

    try:
        os.mkdir('./TrafficSign DataSet/'+imageName_)
    except:
        pass
    SignsDetected = SignDetection(image)
    i = 0
    for [x, y, w, h] in SignsDetected:
        i += 1
        signBoard = image[y:y+h, x:x+w]
        signBoard = imutils.resize(signBoard, width=500)
        greyScaleImage = GrayScaleConverstion(signBoard, greyScaleFilter)
        SaveImage('./TrafficSign DataSet/'+imageName_+ str(i), greyScaleImage)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/" +
        #           "greyScaleImage" + str(i), greyScaleImage)

        # gaussianNoisyImage1 = AddGaussianNoise(greyScaleImage, mean, 1)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/" +
        #           "noice-1-" + str(i), gaussianNoisyImage1)

        # saltPepperNoisyImage10 = AddSaltPepperNoise(greyScaleImage, 10)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/" +
        #           "noice-2-" + str(i), saltPepperNoisyImage10)

        # saltPepperNoisyImage30 = AddSaltPepperNoise(greyScaleImage, 30)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/" +
        #           "noice-3-" + str(i), saltPepperNoisyImage30)

        # boxFilter3x3ImageOnsaltPepperNoisyImage30 = Convolve(
        #     saltPepperNoisyImage30, boxKernel3x3)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-4-" +
        #           str(i), boxFilter3x3ImageOnsaltPepperNoisyImage30)

        # median3x3FilterImageOnSaltPepperNoisyImage30 = MedianFilter(
        #     saltPepperNoisyImage30, 3)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-5-" +
        #           str(i), median3x3FilterImageOnSaltPepperNoisyImage30)

        # gaussian3x3FilterImageOnSaltPepperNoisyImage30 = Convolve(
        #     saltPepperNoisyImage30, gaussianKernel3x3)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-6-" +
        #           str(i), gaussian3x3FilterImageOnSaltPepperNoisyImage30)

        # boxFilter5x5ImageOnsaltPepperNoisyImage30 = Convolve(
        #     saltPepperNoisyImage30, boxKernel5x5)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-7-" +
        #           str(i), boxFilter5x5ImageOnsaltPepperNoisyImage30)

        # median5x5FilterImageOnSaltPepperNoisyImage30 = MedianFilter(
        #     saltPepperNoisyImage30, 5)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-8-" +
        #           str(i), median5x5FilterImageOnSaltPepperNoisyImage30)

        # gaussian5x5FilterImageOnSaltPepperNoisyImage30 = Convolve(
        #     saltPepperNoisyImage30, gaussianKernel5x5)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-9-" +
        #           str(i), gaussian5x5FilterImageOnSaltPepperNoisyImage30)

        # boxFilter9x9ImageOnsaltPepperNoisyImage30 = Convolve(
        #     saltPepperNoisyImage30, boxKernel9x9)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-10-" +
        #           str(i), boxFilter9x9ImageOnsaltPepperNoisyImage30)

        # median9x9FilterImageOnSaltPepperNoisyImage30 = MedianFilter(
        #     saltPepperNoisyImage30, 9)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-11-" +
        #           str(i), median9x9FilterImageOnSaltPepperNoisyImage30)

        # gaussian9x9FilterImageOnSaltPepperNoisyImage30 = Convolve(
        #     saltPepperNoisyImage30, gaussianKernel9x9)
        # SaveImage('./TrafficSign DataSet/'+imageName_+"/"+"noice-12-" +
        #           str(i), gaussian9x9FilterImageOnSaltPepperNoisyImage30)
